In [112]:
import docx
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from  gensim.summarization import keywords
from collections import defaultdict
tokenizer = RegexpTokenizer(r'\w+')

def readtxt(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
doc=readtxt('test1.docx')

In [113]:

def preprocess(sentence):
    sentence = sentence.lower()
    rem_num = re.sub('[0-9]+', '', sentence)#REMOVING NUMBERS
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if len(w) > 5 if not w in stopwords.words('english')]
    return " ".join(filtered_words) 

doc = preprocess(doc) 


In [118]:
#GENISM
values = keywords(text=doc,split=' ',scores=True)
gen = pd.DataFrame(values,columns=['keyword','score'])
gen = gen.sort_values('score',ascending=False)

In [119]:
#FREQUENT WORDS
ngram_freq = nltk.FreqDist() #WE INITIALIZED A FREQUENCY COUNTER

for ngram in nltk.ngrams(tokenizer.tokenize(doc), 1):
    ngram_freq[ngram] += 1
freq_words=ngram_freq.most_common(10) 


In [120]:
#BIGRAMS
words=tokenizer.tokenize(doc)

def ngrams(words, n=2, padding=False):
    grams =  words 
    return (tuple(grams[i:i+n]) for i in range(0, len(grams) - (n - 1)))

counts = defaultdict(int)
for ng in ngrams(words, 2, False):
    counts[ng] += 1
    
bigram=[]
for c, ng in sorted(((c, ng) for ng, c in counts.items()), reverse=True):
    bigram.append(ng)

In [121]:
final=[]
final=pd.DataFrame(final)
freq_words=pd.DataFrame(freq_words)
freq_words[0]=freq_words[0].astype('str')
freq_words['freq_words'] = freq_words[0].map(lambda x: re.sub(r'\W+', '', x))
final['uni-grams']=freq_words['freq_words']
bigram=pd.DataFrame(bigram)
bigram[3]="  "
bigram['bigram'] =bigram[0].map(str) +bigram[3] + bigram[1].map(str)
final['bi-grams']=bigram['bigram'].head(10)
final['genism']=gen['keyword'].head(10)

In [122]:
final

,uni-grams,bi-grams,NMF/LDA
0,spread,performance stress,stressed
1,stress,population performance,evolution
2,evolution,spread favourable,invasion
3,performance,interspecific competition,invasive
4,population,evolution spread,predictions population spread
5,populations,evolutionary change,genotype
6,plants,stressful conditions,genotypes
7,tolerance,favourable habitat,genotyped
8,conditions,population spread,genotypic
9,evolutionary,generations spread,stressful environments
